In [ ]:
import os

In [17]:
# Create proposals json file from ./proposals/*.md files

proposals = []

from pathlib import Path
import re, json

categories = ["unknown", "infrastructure", "integrations", "feature"]

for p in Path('.').glob('proposals/*.md'):
    filename = p.name.replace(".md", "")
    id = filename.replace("wip-", "")
    category = id[:1] # First number in id
    text = p.read_text()
    imagePath = re.search("[\/\w]+\.[a-zA-Z]{3,4}", text).group()
    # WIP-1001 ETH 2.0 support
    title = re.search("\n\#\s+WIP-\d+\s+(.+)\n*", text).group(1)
    body = re.search("\n\#\s+WIP-\d+\s+.+\n*(.*)", text).group(1)

    proposals.append({
        "category": categories[int(category)],
        "id": id,
        "image_url": ("https://raw.githubusercontent.com/sonsofcrypto/web3wallet-improvement-proposals/master"+imagePath),
        "title": title,
        "creation_date": "2022-08-28T00:00:00.000Z",
        "votes": 0,
        "body": body
    })
file = open('proposal-list-new.json', 'w')
file.write(json.dumps(proposals))
file.close()

In [18]:
# Create static page from template each proposal ./static/${PROPOSAL_ID}.html
templateFile = open('./static/template.html', 'r')
template = templateFile.read()
templateFile.close()

for p in proposals:
    t = template
    t = t.replace("${ID}", p["id"])
    t = t.replace("${TITLE}", p["title"])
    t = t.replace("${DESCRIPTION}", "web3Wallet improvement proposal " + p["id"])
    t = t.replace("${IMAGE_URL}", p["image_url"].replace("\\", ""))
    t = t.replace("${CONTENT}", p["body"].replace("\\u", "\\n")) # Picks up \u instead of \n
    t = t.replace("${URL}", "https://sonsofcrypto.com/proposals/"+p["id"]+".html")
    file = open('./static/'+p["id"]+'.html', 'w')
    file.write(t)
    file.close()

In [8]:
# Have to run step one to have proposals in memory
#
# Might be better ways but this is one way to get a Twitter bearer token
# 1. https://oauth-playground.glitch.me/?id=tweetsRecentSearch
# 2. Run a query and authorize with Twitter account
# 3. Click the dots on the right hand side
# 4. Click Include access token
# 5. It should now be visible in the query, just capture the <code>:
# -H "Authorization: Bearer <code>"
# It will expire after 6 hours or something

# Download votes (number of tweets with hashtag for given proposal)
import os, tweepy, time, json
from datetime import datetime, timedelta, date
from dateutil.parser import isoparse
from dotenv import load_dotenv
load_dotenv()


# Get proposals
proposalsFile = open('./proposal-list-new.json', 'r')
proposals = json.loads(proposalsFile.read())
proposalsFile.close()
if type(proposals) is not list:
    exit ("No proposals found")
# Get votes
try:
    votesFile = open('./proposal-votes.json', 'r')
    votes = json.loads(votesFile.read())
    votesFile.close()
except:
    votes = False
    print(">>> No votes file found, creating a new file.")

proposalVotes = []


api_key = os.environ.get("TWITTER_API", "")
api_key_secret = os.environ.get("TWITTER_API_SECRET", "")
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN", "")


if api_key == "" or api_key_secret == "":
    exit("API keys not set for twitter, please set in env TWITTER_API and TWITTER_API_SECRET")

# Connect to twitter API
max_results = 100
sleep_time = 0.5 # seconds

#auth = tweepy.OAuth2BearerHandler(bearer_token);
#api = tweepy.Client(auth)
api = tweepy.Client(bearer_token=bearer_token)

print ("Sleeping "+ str(sleep_time) + " seconds each query")

def find_votes(_proposal):
    if not votes:
        return 0
    for _vote in votes:
        if _vote['id'] == _proposal['id']:
            return _vote
    return 0

for proposal in proposals:
    # Needs to happen in loop since time is moving while executing
    today = datetime.utcnow()
    week_ago = today - timedelta(days=7) + timedelta(minutes=1)

    vote = find_votes(proposal)
    start_time = week_ago
    count = 0

    if vote != 0 and vote and vote['start_time'] and week_ago < isoparse(vote['start_time']):
        start_time = isoparse(vote['start_time'])
        count = vote['count']
        previous_count = vote['count']
    start_time = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')

    hashtag = "#WIP_"+proposal.get("id") #"#WIP_3001"

    next_token = 0
    while next_token != -1:
        if (next_token == 0):
            response = api.search_recent_tweets(query=hashtag, start_time=start_time, max_results=max_results)
        else:
            response = api.search_recent_tweets(query=hashtag, next_token=next_token, max_results=max_results)

        response_count = int(response.meta['result_count'])
        count = count+response_count
        print ("Processing: hashtag: "+hashtag+
               ", count: "+ str(count) + " (" + str(count-previous_count) + " new)" +
               ", start_time: "+ today.strftime('%Y-%m-%dT%H:%M:%SZ')
               )

        next_token = -1
        if response_count == 100 and response.meta['next_token']:
            next_token = response.meta['next_token']

        time.sleep(sleep_time)

    proposalVotes.append({
        "count": count,
        "hashtag": hashtag,
        "start_time": today.isoformat(),
        "id": proposal["id"]
    })

print("Writing file: proposal-votes.json")
file = open('proposal-votes.json', 'w')
file.write(json.dumps(proposalVotes))
file.close()
print("Done.")
# Retrieve twitter posts with hastag

# Count posts

# Apply to correct place


Sleeping 0.5 seconds each query


Unauthorized: 401 Unauthorized
Unauthorized